In [1]:
!pip install requests
!pip install dict2xml

In [2]:
import requests
from lxml import html
from dict2xml import dict2xml

In [3]:
output_file = "./data.xml"
starting_id = 0 # CHNAGE THIS TO LAST ID IN XML FILE

In [4]:
def get_unknown_fields(xpath_query):
    name, value = (t.strip() for t in tree.xpath(f"{xpath_query}/text()")[0].split(": "))
    name = name.lower().replace(" ", "_").replace("-", "_").split("(")[0]
    if not value:
        value = tree.xpath(f"{xpath_query}/a/text()")[0].strip()
    return name, value

In [11]:
# scrapes data from "https://thegamesdb.net/game.php?id={i}" and appends to output_file.xml
with open(output_file, "a", encoding="utf-8") as f:
    f.write("<GAMES>")
    for i in range(starting_id, 10000):
        url = f"https://thegamesdb.net/game.php?id={i}"
        page = requests.get(url)
        tree = html.fromstring(page.text)

        try:
            game_name = tree.xpath("/html/body/div/div[2]/div[2]/div[1]/div/div/div[1]/h1/text()")[0].strip()
            description = tree.xpath("/html/body/div/div[2]/div[2]/div[1]/div/div/div[2]/p[1]/text()")[0].strip()
        except IndexError: continue

        gamesdb_link = url
        esrb_rating = ""
        genres = ""

        for j in range(2,6):
            try:
                name, value = tree.xpath(f"/html/body/div/div[2]/div[2]/div[1]/div/div/div[2]/p[{j}]/text()")[0].split(": ")
                if value:
                        esrb_rating = value if "esrb" in name.lower() else esrb_rating
                        genres = value if "genre" in name.lower() else genres
            except IndexError: continue

        data = {
                "game": {
                        "game_id": i,
                        "game_name": game_name,
                        "description": description,
                        "esrb_rating": esrb_rating,
                        "genres": genres, 
                        "gamesdb_link": gamesdb_link
                        }
                }

        for k in range(2,10):
            try:
                xpath_query = f"/html/body/div/div[2]/div[1]/div/div/div/div[2]/p[{k}]"
                name, value = get_unknown_fields(xpath_query)
                data["game"][name] = value
            except IndexError: continue

        xml = dict2xml(data, indent=False)
        f.write(xml)
        # print(xml, sep="\n----\n")
        print(i)
    f.write("</GAMES>")

1
2
4
5
6
9


In [ ]:
# i=9000
# url = f"https://thegamesdb.net/game.php?id={i}"
# page = requests.get(url)
# tree = html.fromstring(page.text)

# game_name = tree.xpath("/html/body/div/div[2]/div[2]/div[1]/div/div/div[1]/h1/text()")[0]
# description = tree.xpath("/html/body/div/div[2]/div[2]/div[1]/div/div/div[2]/p[1]/text()")[0]
# esrb_rating = ""
# genres = ""

# for k in range(2,6):
#     try:
#         xpath_query = f"/html/body/div/div[2]/div[1]/div/div/div/div[2]/p[{k}]"
#         name, value = tree.xpath(f"{xpath_query}/text()")[0].split(": ")
#         name = name.lower().replace(" ", "_").strip().split("(")[0]
#         value = value.strip()
#         if not value:
#             value = tree.xpath(f"{xpath_query}/a/text()")[0]
        
#         # /html/body/div/div[2]/div[1]/div/div/div/div[2]/p[1]/a
#         print(name, [value], value)
#     except IndexError: continue

region ['Region Not Set'] Region Not Set
publishers ['Konami'] Konami
releasedate ['2011-11-22'] 2011-11-22
co-op ['No'] No
